In [11]:
import numpy as np
import pandas as pd
import re

### useful funcs

In [113]:
def find_words(str):
    pattern = r'\b[A-Za-z\']+\b'
    full_words = re.findall(pattern, str)
    return np.array(full_words)

### loading data and feature selection 

In [114]:
#loading in data and getting list of words per record
review_train_data = pd.read_csv("movie_review.csv",encoding= 'unicode_escape')
review_train_data['words'] = review_train_data['Review'].apply(find_words)
review_train_data

,Review,Polarity,words
0,summer catch ( 2001 ) . starring freddie prinz...,-1,"[summer, catch, starring, freddie, prinze, jr,..."
1,as you should know this summer has been less ...,-1,"[as, you, should, know, this, summer, has, bee..."
2,back in the early years of his career eddie m...,-1,"[back, in, the, early, years, of, his, career,..."
3,the modern use of the western movie and the w...,-1,"[the, modern, use, of, the, western, movie, an..."
4,note : some may consider portions of the follo...,-1,"[note, some, may, consider, portions, of, the,..."
...,...,...,...
195,er is een leuke vondst in deze film . in het a...,1,"[er, is, een, leuke, vondst, in, deze, film, i..."
196,steven spielberg is now considered as one of t...,1,"[steven, spielberg, is, now, considered, as, o..."
197,member of the 'internet movie critic's associa...,1,"[member, of, the, internet, movie, critic's, a..."
198,after watching the first ten minutes of this j...,1,"[after, watching, the, first, ten, minutes, of..."


In [150]:
#get all positive words 
train_pos = review_train_data[review_train_data["Polarity"] == 1]['words'].to_list()
pos_words = np.array([])
for word_arr in train_pos:
    unique_lst = list(set(word_arr))
    pos_words = np.append(pos_words, unique_lst)
pos_words_set = set(pos_words)
unique_pos_words, pos_word_count =np.unique(pos_words, return_counts= True)
pos_df = pd.DataFrame({'words': unique_pos_words, 'count': pos_word_count})
pos_df

,words,count
0,'s,19
1,a,98
2,aan,1
3,aangekomen,1
4,abandon,3
...,...,...
10430,zoolander's,1
10431,zorgt,1
10432,zuko,1
10433,zwick,1


In [151]:
#get all negative words
train_neg = review_train_data[review_train_data["Polarity"] == -1]['words'].to_list()
neg_words = np.array([])
for word_arr in train_neg:
    unique_lst = list(set(word_arr))
    neg_words  = np.append(neg_words , unique_lst)
neg_words_set  = set(neg_words )
unique_neg_words, neg_word_count =np.unique(neg_words, return_counts= True)
neg_df = pd.DataFrame({'words': unique_neg_words, 'count': neg_word_count})
neg_df

,words,count
0,',2
1,'s,20
2,a,98
3,aa,1
4,aback,1
...,...,...
9495,zippers,1
9496,zombies,1
9497,zone,1
9498,zwick,1


In [153]:
only_pos = list(pos_words_set - neg_words_set)
only_neg = list(neg_words_set - pos_words_set)
len(only_pos), len(only_neg)

(5360, 4425)

In [158]:
only_pos_df = pos_df[np.isin(pos_df['words'], only_pos)]
only_pos_df.sort_values(by='count', ascending=False).head(15)

,words,count
7196,professional,14
3066,era,9
1690,commentary,8
2387,detailed,7
2386,detail,7
9469,toward,7
23,absolute,6
2863,effectively,6
6343,notch,6
6250,nevertheless,6


In [159]:
only_neg_df = neg_df[np.isin(neg_df['words'], only_neg)]
only_neg_df.sort_values(by='count', ascending=False).head(15)

,words,count
3205,fine,9
4760,lame,9
590,awful,8
4908,lifeless,8
621,badly,7
190,alas,7
8886,uninspired,7
6939,remotely,7
8378,tedious,6
1044,broken,6
